In [ ]:
reload_lamb()

# Introduction to type shifting in ILNB
### Notebook author: Kyle Rawlins

This notebook goes through the basics of how to do type-shifting in the lambda notebook.  Basically, you add a unary rule to the composition system, designate it as a typeshift, and enable type-shifting.  Currently, type-shifting is purely as a last resort.  (If you need to simulate other cases, it is best right now just to use unary rules or extra lexical entries.)

In general, I highly recommend that type-shifting be implemented via *combinators*.  A combinator is simply any function that has no free variables.  To build a typeshifting combinator, simply take the typeshift and use the shifted type as a further lambda term.  This strategy will make your life much easier, as it relies only on functional application to do its work.  The alternative, which may be helpful in certain special cases (not documented here) would be to write python code that performs the shift.

For example, here is a standard way of defining a type-lifting rule from Partee:

 * lift: $x \rightarrow \lambda f_{<e,t>} : f(x)$
 
The combinator for this typeshift is:

In [ ]:
%%lamb
gqlift = L x_e : L f_<e,t> : f(x)

A combinator like this can be defined as a ILNB meta-language expression, and conveniently also can be called to perform the type-shift, like any python callable.

In [ ]:
%lamb thecat = Iota x_e : Cat_<e,t>(x)
gqlift(thecat).reduce_all()

(All of the combinators I discuss here are of fixed type.  For flexibly typed combinators, see the examples in the "Variable free binding" notebook.)

### Predicate Modification (PM via type-shifting

The normal PM rule is already implemented in the lambda notebook via a combinator.  This means it is a good candidate for a type-shift.

In [ ]:
%%lamb
||gray|| = L x_e : Gray(x)
||cat|| = L x_e : Cat(x)

In [ ]:
(gray * cat).tree()

In [ ]:
pm_combinator = te("L f_<e,t> : L g_<e,t> : L x_e : f(x) & g(x)")
pm_combinator

The following code creates a composition system with the PM-shift instead of the regular PM operation.

Note the neat trick whereby the combinator can be used directly as the typeshift function.  You won't always want to do this, but it is very clean when it works.  It works because though the combinator is a TypedExpr, `__call__` is defined and so it can be treated like any other python function with one argument (exactly what the factory is looking for).  The reduction step is applied by the object constructed from this factory, and can be disabled with a keyword argument.

In [ ]:
system = lang.td_system.copy()
system.add_rule(lang.unary_factory(pm_combinator, "PM-shift", typeshift=True)) # add this as a typeshift
system.remove_rule("PM") # remove ordinary PM
system.typeshift = True # enable typeshifts in the composition system
lang.set_system(system) # set the new system as the default
system

In [ ]:
r = (gray * cat)
r

In [ ]:
r.tree()

### Quantifiers in object position

The "standard" approach to quantifiers in object position is to assume that a type mismatch can trigger QR.  However, an alternative is to type-shift the DP to a type where it can take a transitive predicate and ignore the external argument position.

This would need to be generalized for n-ary predicates.

In [ ]:
%%lamb
||every|| = L f_<e,t> : L g_<e,t> : Forall x_e : f(x) >> g(x)
||doctor|| = L x_e : Doctor(x)
||someone|| = L f_<e,t> : Exists x_e : Human(x) & f(x)
||saw|| = L x_e : L y_e : Saw(y,x)
||alfonso|| = Alfonso_e

In [ ]:
((every * doctor) * (saw * alfonso)).tree()

In [ ]:
(saw * (every * doctor))

The following combinator shifts a GQ type into something that can handle a transitive verb.

In [ ]:
gq_lift_combinator = %te L f_<<e,t>,t> : L g_<e,<e,t>> : L x_e : f(L y_e : g(y)(x))
gq_lift_combinator

In [ ]:
gq_lift_combinator(someone.content).reduce_all()

In [ ]:
system = lang.td_system.copy()
system.add_rule(lang.unary_factory(gq_lift_combinator, "gq-lift-trans", typeshift=True))
system.typeshift = True
lang.set_system(system)
system

In [ ]:
(alfonso * (saw * someone))

In [ ]:
(someone * (saw * (every * doctor)))

In [ ]:
r = ((every * doctor) * (saw * someone))
r

In [ ]:
r.tree()

This so far produces only surface scope readings when there are multiple quantifiers.

Following work in CCG, one might imagine that composition needn't match constituency; if the subject shifts and composes with the verb before the object we can get the other scoping.  (In CCG this is implemented using a function composition operation, not a type-shift.)

In [ ]:
r = (someone * (saw * (every * doctor)))
r

Someone interested in constituency might find this unsatisfying.  How could this be resolved using a type-shift?  One idea (due to Hendriks) is to build scope-taking shifts that operate on verb meanings.  This is not necessarily more satisfying..

In [ ]:
surface_shift_comb = te("L v_<e,<e,t>> : L f_<<e,t>,t> : L g_<<e,t>,t> : g(L y_e : f(L x_e : (v(x)(y))))")
inverse_shift_comb = te("L v_<e,<e,t>> : L f_<<e,t>,t> : L g_<<e,t>,t> : f(L x_e : g(L y_e : (v(x)(y))))")

inverse_shift_comb(saw.content).reduce_all()

In [ ]:
system = lang.td_system.copy()
system.add_rule(lang.unary_factory(surface_shift_comb, "surface", typeshift=True))
system.add_rule(lang.unary_factory(inverse_shift_comb, "inverse", typeshift=True))
system.typeshift = True
lang.set_system(system)
system

In [ ]:
r = (someone * ((every * doctor) * saw))
r

In [ ]:
r[1].tree()

A final strategy would be to provide an even higher object type-lift that implements inverse scope.  This is effectively the combinator for Hendriks' inverse scope shifter with the order of arguments reversed.

In [ ]:
gq_lift_combinator = te("L f_<<e,t>,t> : L g_<e,<e,t>> : L x_e : f(L y_e : g(y)(x))")
gq_lift_combinator2 = te("L f_<<e,t>,t> : L g_<e,<e,t>> : L h_<<e,t>,t> : f(L y_e : h(L x_e : g(y)(x)))")

gq_lift_combinator2.type

In [ ]:
system = lang.td_system.copy()
system.add_rule(lang.unary_factory(gq_lift_combinator, "gq-lift-trans", typeshift=True))
system.add_rule(lang.unary_factory(gq_lift_combinator2, "gq-lift2-trans", typeshift=True))
system.typeshift = True
lang.set_system(system)
system

In [ ]:
r = (someone * ((every * doctor) * saw))
r

In [ ]:
r.tree()

### Partee's type-shifts

(needs to be added)